In [ ]:
import numpy as np
import open3d as o3d

from lead.common import common_utils
from lead.common.constants import (
    SEMANTIC_SEGMENTATION_CONVERTER,
    TRANSFUSER_SEMANTIC_COLORS,
    CameraPointCloudIndex,
    CarlaSemanticSegmentationClass,
    TransfuserSemanticSegmentationClass,
)

In [ ]:
# Load pre-computed camera point cloud and bounding boxes
frame_id = 69
root = "data/expert_debug/data/ConstructionObstacle/Town03_Rep-1_route_001977_route0_01_14_22_27_13"

pc_path = f"{root}/camera_pc/{str(frame_id).zfill(4)}.npz"
bb_path = f"{root}/bboxes/{str(frame_id).zfill(4)}.pkl"

points_ego = np.load(pc_path)["arr_0"]
boxes = common_utils.read_pickle(bb_path)

print(f"Point cloud shape: {points_ego.shape}")
print(f"Number of bounding boxes: {len(boxes)}")

In [ ]:
# 3D visualization with Open3D
pcd = o3d.geometry.PointCloud()

# Ensure points_ego is in the correct shape (N, 3) and dtype
if points_ego.shape[0] == 3 or points_ego.shape[0] == 4:
    # If shape is (3, N) or (4, N), transpose to (N, 3) or (N, 4)
    points_ego = points_ego.T

# Extract xyz coordinates and semantic class using the proper indices
points_xyz = points_ego[:, [CameraPointCloudIndex.X, CameraPointCloudIndex.Y, CameraPointCloudIndex.Z]].astype(np.float64)
carla_semantic_classes = points_ego[:, CameraPointCloudIndex.UNREAL_SEMANTICS_ID].astype(np.int32)

print(f"Unique CARLA semantic classes in data: {np.unique(carla_semantic_classes)}")

# Convert CARLA semantic classes to TransFuser semantic classes
transfuser_semantic_classes = np.array(
    [
        SEMANTIC_SEGMENTATION_CONVERTER.get(CarlaSemanticSegmentationClass(c), TransfuserSemanticSegmentationClass.UNLABELED)
        for c in carla_semantic_classes
    ],
    dtype=np.int32,
)

print(f"Unique TransFuser semantic classes: {np.unique(transfuser_semantic_classes)}")

# Filter out points that are too far away (> 100m)
distances = np.linalg.norm(points_xyz, axis=1)
mask = distances <= 100.0
points_xyz = points_xyz[mask]
transfuser_semantic_classes = transfuser_semantic_classes[mask]
print(f"Points after filtering (≤100m): {points_xyz.shape[0]}")

# Convert TransFuser colors from RGB (0-255) to normalized [0-1] for Open3D
TRANSFUSER_SEMANTIC_COLORS[TransfuserSemanticSegmentationClass.UNLABELED] = [125, 125, 125]  # Grey unlabeled
transfuser_colors_normalized = {k: [v[0] / 255.0, v[1] / 255.0, v[2] / 255.0] for k, v in TRANSFUSER_SEMANTIC_COLORS.items()}

# Assign colors to points based on TransFuser semantic class
colors = np.array(
    [
        transfuser_colors_normalized.get(
            TransfuserSemanticSegmentationClass(int(c)),
            [0.0, 0.0, 0.0],  # Black for unlabeled
        )
        for c in transfuser_semantic_classes
    ]
)

pcd.points = o3d.utility.Vector3dVector(points_xyz)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Create bounding box visualizations
geometries = [pcd]

for box in boxes:
    # Create oriented bounding box
    bbox = o3d.geometry.OrientedBoundingBox()

    # Set center position
    bbox.center = box["position"]

    # Set extent (size) - multiply by 2 to convert from half-extent to full extent
    bbox.extent = [2 * e for e in box["extent"]]

    # Set rotation from yaw angle (rotation around z-axis)
    yaw = box["yaw"]
    R = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])
    bbox.R = R

    # Color based on class
    if box.get("class") == "ego_car":
        bbox.color = [0, 1, 0]  # Green for ego
    elif box.get("transfuser_semantics_id") == CarlaSemanticSegmentationClass.Car:
        bbox.color = [1, 0, 0]  # Red for vehicles
    elif box.get("transfuser_semantics_id") == CarlaSemanticSegmentationClass.Pedestrian:
        bbox.color = [0, 0, 1]  # Blue for pedestrians
    else:
        bbox.color = [1, 1, 0]  # Yellow for others

    geometries.append(bbox)

o3d.visualization.draw_geometries(geometries, window_name="Point Cloud + Bounding Boxes")